# 使用2022年的数据训练决策树

2022年的主要改动在于：提供了非常多的固定动量p的正电子的模拟数据，并且没有给出波形，而是给出了PETime。后者是一个好消息，意味着2021年的手作算法彻底退出舞台。
这里我们先按照去年一样的特征工程，即五个特征：

1. 单个事件的PE总数；
2. 单个事件，对于每个有PE的PMT，每个PMT会对应一个PE数。每个事件都有一个这样的序列，这个序列的平均值；
3. 2中序列的标准差；
4. 单个事件，对于每个有PE的PMT，每个PMT会对应一系列PETime。先对单个PMT进行PETime平均，然后这个PETime平均对于每个PMT构成一个序列，这个序列的平均值；
5. 4中序列的标准差。

今年没有暗噪声，因此代码会相当直接。

In [2]:
import numpy as np
import h5py
import lightgbm as lgb
from tqdm import tqdm
import multiprocessing

## 读取数据集
我们可能需要区分固定动量正电子和非固定动量正电子的训练数据，因此这里分开读。为了保持与去年的一致性，分开读之后会合并起来。

In [6]:
# 含有10000的是固定动量正电子的数据
# 结果证明固定动量也不是完全固定，可能是因为误差
event_count_10000 = np.empty(11, dtype=int)
PETruth_entries_10000 = np.empty(11, dtype=int)
for data_id in range(11):
    with h5py.File(f'data-2022/{data_id}-10000-exact.h5', 'r') as data_file:
        event_count_10000[data_id] = data_file['ParticleTruth'].shape[0]
        PETruth_entries_10000[data_id] = data_file['PETruth'].shape[0]


p_10000 = np.empty(event_count_10000.sum())

event_count_index_10000 = np.insert(np.cumsum(event_count_10000), 0, 0)
PETruth_entries_index_10000 = np.insert(np.cumsum(PETruth_entries_10000), 0, 0)

PETruth_10000 = np.empty(PETruth_entries_10000.sum(), dtype=[('EventID', '<i4'), ('ChannelID', '<i4'), ('PETime', '<f8')])
for data_id in tqdm(range(11)):
    with h5py.File(f'data-2022/{data_id}-10000-exact.h5', 'r') as data_file:
        p_10000[event_count_index_10000[data_id]:event_count_index_10000[data_id+1]] = data_file['ParticleTruth']['p'][...]

        single_PETruth = data_file['PETruth'][...]
        previous_events = event_count_index_10000[data_id]
        single_PETruth['EventID'] += previous_events
        PETruth_10000[PETruth_entries_index_10000[data_id]:PETruth_entries_index_10000[data_id+1]] = single_PETruth

100%|██████████| 11/11 [01:59<00:00, 10.86s/it]


In [5]:
# 不含有10000的是随机动量正电子的数据
event_count = np.empty(20, dtype=int)
PETruth_entries = np.empty(20, dtype=int)
for data_id in range(20):
    with h5py.File(f'data-2022/{data_id}-exact.h5', 'r') as data_file:
        event_count[data_id] = data_file['ParticleTruth'].shape[0]
        PETruth_entries[data_id] = data_file['PETruth'].shape[0]


p = np.empty(event_count.sum())

event_count_index = np.insert(np.cumsum(event_count), 0, 0)
PETruth_entries_index = np.insert(np.cumsum(PETruth_entries), 0, 0)

PETruth = np.empty(PETruth_entries.sum(), dtype=[('EventID', '<i4'), ('ChannelID', '<i4'), ('PETime', '<f8')])
for data_id in tqdm(range(20)):
    with h5py.File(f'data-2022/{data_id}-exact.h5', 'r') as data_file:
        p[event_count_index[data_id]:event_count_index[data_id+1]] = data_file['ParticleTruth']['p'][...]

        single_PETruth = data_file['PETruth'][...]
        previous_events = event_count_index[data_id]
        single_PETruth['EventID'] += previous_events
        PETruth[PETruth_entries_index[data_id]:PETruth_entries_index[data_id+1]] = single_PETruth

100%|██████████| 20/20 [01:58<00:00,  5.91s/it]


笑死，这样也太占内存了。其实我们完全没有必要把东西都读出来，边读边处理它不香吗？
## 数据处理，得到五个特征

In [3]:
# 先看看训练集到底有多少个事件
# 我们总共有31个训练集，11个固定动量，20个随机动量。这些就hard-code了

event_count = np.zeros(31, dtype=int)

for data_id in range(11):
    with h5py.File(f'data-2022/{data_id}-10000-exact.h5', 'r') as data_file:
        event_count[data_id] = data_file['ParticleTruth'].shape[0]

for data_id in range(20):
    with h5py.File(f'data-2022/{data_id}-exact.h5', 'r') as data_file:
        event_count[data_id+11] = data_file['ParticleTruth'].shape[0]

event_total = event_count.sum()
print(f'训练集一共有{event_total}个事件')

训练集一共有310000个事件


好的，我们看到这次的训练集没有像去年一样整什么“事件消失术”。接下来直接上特征。

`PE_total`: 单次事件的PE总数

`PE_mean`: 单次事件所有有PE的PMT的PE数平均值

`PE_std`: 单次事件所有有PE的PMT的PE数标准差

`PETime_mean`: 单次事件，PETime对单个PMT取平均值，再对所有有PE的PMT取平均值

`PETime_std`: 单次事件，PETime对单个PMT取平均值，再对所有有PE的PMT取标准差

In [4]:
# 训练目标
p_train = np.zeros(event_total)

# 特征
PE_total_train = np.zeros(event_total, dtype='<i4')
PE_mean_train = np.zeros(event_total, dtype='<f8')
PE_std_train = np.zeros(event_total, dtype='<f8')
PETime_mean_train= np.zeros(event_total, dtype='<f8')
PETime_std_train = np.zeros(event_total, dtype='<f8')

In [9]:
event_index_total = np.insert(np.cumsum(event_count), 0, 0)

for data_id in range(11):
    with h5py.File(f'data-2022/{data_id}-10000-exact.h5', 'r') as data_file:
        p_train[event_index_total[data_id]:event_index_total[data_id+1]] = data_file['ParticleTruth']['p'][...]

        PETruth = data_file['PETruth'][...]

        # 先按照event分组，得到PE总数
        event_ids, event_indices, event_PE_counts = np.unique(PETruth['EventID'], return_index=True, return_counts=True)
        PE_total_train[event_index_total[data_id]:event_index_total[data_id+1]] = event_PE_counts

        # 分组过后，分单个event进行处理
        event_indices = np.append(event_indices, PETruth.shape[0]+1)

        # 定义一个函数，用于多线程处理单个事件。每个线程是独立的，只是给一个数组赋值，没有输出
        def deal_with_events(event_id):
            event_PETruth = PETruth[event_indices[event_id]:event_indices[event_id+1]]

            # 对ChannelID再次进行分组
            channel_ids, channel_inverse, channel_PE_counts = np.unique(event_PETruth['ChannelID'], return_inverse=True, return_counts=True)

            # 得到PE_mean_train与PE_std_train
            PE_mean_train_single = channel_PE_counts.mean()
            PE_std_train_single = channel_PE_counts.std()

            # 拿出单个channel的信息
            channel_PETime_mean = np.zeros(channel_ids.shape[0])
            # channel_indices = np.append(channel_indices, channel_ids.shape[0]+1)
            for channel_number in range(channel_ids.shape[0]):
                channel_PETime_mean[channel_number] = event_PETruth['PETime'][
                    channel_inverse == channel_number
                ].mean()
            
            # 得到PETime_mean_train与PETime_std_train
            PETime_mean_train_single = channel_PETime_mean.mean()
            PETime_std_train_single = channel_PETime_mean.std()

            return PE_mean_train_single, PE_std_train_single, PETime_mean_train_single, PETime_std_train_single
        
        with multiprocessing.Pool(20) as p:
            res = np.array(list(tqdm(p.imap(deal_with_events, event_ids), total=event_ids.shape[0])))
            PE_mean_train[event_index_total[data_id]:event_index_total[data_id+1]] = res[:, 0]
            PE_std_train[event_index_total[data_id]:event_index_total[data_id+1]] = res[:, 1]
            PETime_mean_train[event_index_total[data_id]:event_index_total[data_id+1]] = res[:, 2]
            PETime_std_train[event_index_total[data_id]:event_index_total[data_id+1]] = res[:, 3]
        
for data_id in range(20):
    with h5py.File(f'data-2022/{data_id}-exact.h5', 'r') as data_file:
        p_train[event_index_total[data_id+11]:event_index_total[data_id+12]] = data_file['ParticleTruth']['p'][...]

        PETruth = data_file['PETruth'][...]

        # 先按照event分组，得到PE总数
        event_ids, event_indices, event_PE_counts = np.unique(PETruth['EventID'], return_index=True, return_counts=True)
        PE_total_train[event_index_total[data_id+11]:event_index_total[data_id+12]] = event_PE_counts

        # 分组过后，分单个event进行处理
        event_indices = np.append(event_indices, PETruth.shape[0]+1)

        # 定义一个函数，用于多线程处理单个事件。每个线程是独立的，只是给一个数组赋值，没有输出
        def deal_with_events(event_id):
            event_PETruth = PETruth[event_indices[event_id]:event_indices[event_id+1]]

            # 对ChannelID再次进行分组
            channel_ids, channel_inverse, channel_PE_counts = np.unique(event_PETruth['ChannelID'], return_inverse=True, return_counts=True)

            # 得到PE_mean_train与PE_std_train
            PE_mean_train_single = channel_PE_counts.mean()
            PE_std_train_single = channel_PE_counts.std()

            # 拿出单个channel的信息
            channel_PETime_mean = np.zeros(channel_ids.shape[0])
            # channel_indices = np.append(channel_indices, channel_ids.shape[0]+1)
            for channel_number in range(channel_ids.shape[0]):
                channel_PETime_mean[channel_number] = event_PETruth['PETime'][
                    channel_inverse == channel_number
                ].mean()
            
            # 得到PETime_mean_train与PETime_std_train
            PETime_mean_train_single = channel_PETime_mean.mean()
            PETime_std_train_single = channel_PETime_mean.std()

            return PE_mean_train_single, PE_std_train_single, PETime_mean_train_single, PETime_std_train_single
        
        with multiprocessing.Pool(20) as p:
            res = np.array(list(tqdm(p.imap(deal_with_events, event_ids), total=event_ids.shape[0])))
            PE_mean_train[event_index_total[data_id+11]:event_index_total[data_id+12]] = res[:, 0]
            PE_std_train[event_index_total[data_id+11]:event_index_total[data_id+12]] = res[:, 1]
            PETime_mean_train[event_index_total[data_id+11]:event_index_total[data_id+12]] = res[:, 2]
            PETime_std_train[event_index_total[data_id+11]:event_index_total[data_id+12]] = res[:, 3]

100%|██████████| 10000/10000 [00:37<00:00, 266.56it/s]


存储一下得到的特征工程

In [10]:
char_dtype = np.dtype(
    [
        ('PE_total', '<i4'),
        ('PE_mean', '<f8'),
        ('PE_std', '<f8'),
        ('PETime_mean', '<f8'),
        ('PETime_std', '<f8'),
        ('p', '<f8')
    ]
)

char_data = np.zeros(p_train.shape, dtype=char_dtype)
char_data['PE_total'] = PE_total_train
char_data['PE_mean'] = PE_mean_train
char_data['PE_std'] = PE_std_train
char_data['PETime_mean'] = PETime_mean_train
char_data['PETime_std'] = PETime_std_train
char_data['p'] = p_train

with h5py.File(f'train-2022/character.h5', 'a') as char_file:
    char_file.create_dataset("characters", data=char_data)

## 训练模型
我们使用Lightgbm直接训练一个到p的决策树。先写一个损失函数：

In [32]:
def lossfunc_eval(y, data):
    '''
    lossfunc_eval: 用平台的评分标准来评估决策树
    '''
    momentum_true = data.get_label()
    mass = 0.511
    energy_true = np.sqrt(momentum_true**2+mass**2)
    energy_y = np.sqrt(y**2+mass**2)

    loss = (energy_y - energy_true) ** 2 / energy_true
    return "resolution", np.mean(loss), False

def lossfunc_train(y, data):
    '''
    lossfunc_train: 用平台的评分标准算梯度与hessian矩阵
    '''
    momentum_true = data.get_label()
    mass = 0.511
    energy_true = np.sqrt(momentum_true**2+mass**2)
    energy_y = np.sqrt(y**2+mass**2)

    grad = 2*(y-y*np.sqrt(momentum_true**2+mass**2)/np.sqrt(y**2+mass**2)) / energy_true
    hess = 2*(1 - y*energy_true/energy_y)/ energy_true
    return grad, hess

In [35]:
train_data_p = lgb.Dataset(
    np.stack(
        (PE_total_train[:-30000], PE_mean_train[:-30000], PE_std_train[:-30000], PETime_mean_train[:-30000], PETime_std_train[:-30000]),
        axis=1
    ),
    label=p_train[:-30000]
)

validation_data_p = lgb.Dataset(
    np.stack(
        (PE_total_train[-30000:], PE_mean_train[-30000:], PE_std_train[-30000:], PETime_mean_train[-30000:], PETime_std_train[-30000:]),
        axis=1
    ),
    label=p_train[-30000:],
    reference=train_data_p
)

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'resolution'},
    'num_leaves': 2**10,
    'learning_rate': 0.01,
    'feature_fraction': 1,
    'bagging_fraction': 1,
    'bagging_freq': 5,
    'verbose': 0,
    'num_threads': 20,
    'max_depth': 10,
}

gbmForP = lgb.train(
    params,
    train_data_p,
    num_boost_round=60000,
    valid_sets=validation_data_p,
    early_stopping_rounds=1000,
    # fobj=lossfunc_train,
    feval=lossfunc_eval
)
gbmForP.save_model('./model/model-2022.txt')

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004244 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's resolution: 1.65164
Training until validation scores don't improve for 1000 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's resolution: 1.61805
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's resolution: 1.58514
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's resolution: 1.55289
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	valid_0's resolution: 1.5213
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[6]	valid_0's resolution: 1.49035
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[7]	valid_0's resolution: 1.46002
[LightGBM]

## 解题

In [18]:
with h5py.File(f'data-2022/problem.h5', 'r') as problem_file:

    PETruth = problem_file['PETruth'][...]

    # 先按照event分组，得到PE总数
    event_ids, event_indices, event_PE_counts = np.unique(PETruth['EventID'], return_index=True, return_counts=True)
    PE_total_problem = event_PE_counts

    # 分组过后，分单个event进行处理
    event_indices = np.append(event_indices, PETruth.shape[0]+1)

    # 定义一个函数，用于多线程处理单个事件。每个线程是独立的，只是给一个数组赋值，没有输出
    def deal_with_events(event_id):
        event_PETruth = PETruth[event_indices[event_id]:event_indices[event_id+1]]

        # 对ChannelID再次进行分组
        channel_ids, channel_inverse, channel_PE_counts = np.unique(event_PETruth['ChannelID'], return_inverse=True, return_counts=True)

        # 得到PE_mean_train与PE_std_train
        PE_mean_train_single = channel_PE_counts.mean()
        PE_std_train_single = channel_PE_counts.std()

        # 拿出单个channel的信息
        channel_PETime_mean = np.zeros(channel_ids.shape[0])
        # channel_indices = np.append(channel_indices, channel_ids.shape[0]+1)
        for channel_number in range(channel_ids.shape[0]):
            channel_PETime_mean[channel_number] = event_PETruth['PETime'][
                channel_inverse == channel_number
            ].mean()
        
        # 得到PETime_mean_train与PETime_std_train
        PETime_mean_train_single = channel_PETime_mean.mean()
        PETime_std_train_single = channel_PETime_mean.std()

        return PE_mean_train_single, PE_std_train_single, PETime_mean_train_single, PETime_std_train_single
    
    with multiprocessing.Pool(20) as p:
        res = np.array(list(tqdm(p.imap(deal_with_events, event_ids), total=event_ids.shape[0])))
        PE_mean_problem = res[:, 0]
        PE_std_problem = res[:, 1]
        PETime_mean_problem = res[:, 2]
        PETime_std_problem = res[:, 3]

100%|██████████| 20000/20000 [01:50<00:00, 181.78it/s]


In [36]:
gbmForP = lgb.Booster(model_file='./model/model-2022.txt')

ans_p = gbmForP.predict(
    np.stack(
        (PE_total_problem, PE_mean_problem, PE_std_problem, PETime_mean_problem, PETime_std_problem),
        axis=1
    )
)

ans_dtype = np.dtype(
    [
        ('EventID', '<i4'),
        ('p', '<f8')
    ]
)

ans_data = np.zeros(ans_p.shape, dtype=ans_dtype)
ans_data['EventID'] = np.arange(ans_p.shape[0])
ans_data['p'] = ans_p


with h5py.File(f'ans-2022/ans_last_year.h5', 'a') as ans_file:
    ans_file.create_dataset("Answer", data=ans_data)

这样评测平台给出的是0.039/0.01。